# Исследование надежности заемщиков.

Банк поставил задачу: разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные — статистика о платёжеспособности клиентов.

**Цель исследования** — проверить четыре гипотезы:
1. Существует зависимость между количеством детей и возвратом кредита в срок.
2. Существует зависимость между семейным положением и возвратом кредита в срок.
3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
4. Разные цели кредита по-разному влияют на его возврат в срок.

**Ход исследования**

Перед проверкой гипотез понадобится обзор данных. 

Необходимо проверить данные на ошибки и оценить их влияние на исследование. Затем, на этапе предобработки мы исправим критичные ошибки данных, устраним дубликаты, обработаем артефакты, категоризируем данные и проверим гипотезы.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


### Шаг 1. Обзор данных

Изучим входные данные от банка — статистику о платёжеспособности клиентов.

In [2]:
import pandas as pd
clients = pd.read_csv('/datasets/data.csv')
display(clients.head(10)) # выведем 10 первых строк таблицы
clients.info() # получим общую информацию о данных в таблице
clients.isna().mean() # проверим наличие дубликатов

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Выводы**

В каждой строке таблицы — данные о заемщике: его/ее возраст, количество детей, пол, образование, трудовой стаж, уровень дохода, тип занятости, наличие просрочек по выплатам кредита в прошлом. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, артефакты, а также явные и неявные дубликаты.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

### Шаг 2. Предобработка данных
- В таблице обнаружены пропущенные значения в столбцах `days_employed` и `total_income`
- Доля пропущенных значений в каждом из столбцов с пропусками составляет 10%.
- Количество пропущенных значений в обоих столбцах одинаково. Из этого можно сделать вывод, что данные по доходу рассчитаны с использованием данных об общем трудовом стаже в днях, данные о которых, вероятно, не были предоставлены клиентами.
- Заполнить пропуски медианным значением — лучшее решение для количественных переменных, потому что среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства, как в приведенных данных.

### Шаг 2.1 Заполнение пропусков
Заполним пропуски в столбце `total_income` медианным значением.

In [3]:
clients['total_income'] = clients['total_income'].fillna(clients['total_income'].median())
clients['total_income'].isna().sum() # проверяем результат


0

### Шаг 2.2 Проверка данных на аномалии и исправления.
В столбце `days_employed` есть отрицательные значения дней трудового стажа. 
Возможные причины появления таких данных:
- человеческий фактор
- сбой при выгрузке данных

Изменим отрицательные числа на их значение по модулю.

In [4]:
clients['days_employed'] = clients['days_employed'].abs() # приведем отрицательные числа к абсолютному значению
clients['days_employed'].sort_values(ascending=False) # отсортируем данные в столбце

6954     401755.400475
10006    401715.811749
7664     401675.093434
2156     401674.466633
7794     401663.850046
             ...      
21489              NaN
21495              NaN
21497              NaN
21502              NaN
21510              NaN
Name: days_employed, Length: 21525, dtype: float64

В некоторых ячейках количество дней не соответствует реальным цифрам, 401 755 рабочих дней — это 1100 лет. Проверим максимальный возраст заемщиков из списка.

In [5]:
clients.loc[clients['dob_years'] == clients['dob_years'].max()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8880,0,1678.969771,75,среднее,1,вдовец / вдова,2,F,госслужащий,0,153282.648133,заняться образованием


При условии, что данный человек начал работать в 18 лет, максимальное количество трудовых дней должно быть не больше (75-18)*365 = 20805 дней.

In [6]:
# используем функцию для замены значений, выходящих за этот предел. Будем считать, что каждый человек из списка начал работать в 18 лет

def days_employed_anomaly(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    max_days_employed = 20805
    if (days_employed > max_days_employed) & (dob_years != 0):
        days_employed = (dob_years - 18) * 365
    if (days_employed > max_days_employed) & (dob_years == 0):
        days_employed = max_days_employed
    return days_employed

clients['days_employed'] = clients.apply(days_employed_anomaly, axis=1)

clients['days_employed'].sort_values(ascending=False) # проверим максимальное значение в столбце 'days_employed'


578      20805.0
1175     20805.0
14659    20805.0
8061     20805.0
12729    20805.0
          ...   
21489        NaN
21495        NaN
21497        NaN
21502        NaN
21510        NaN
Name: days_employed, Length: 21525, dtype: float64

Заполним пропуски в `days_employed` медианными значениями по этому столбцу.

In [7]:
clients['days_employed'] = clients['days_employed'].fillna(clients['days_employed'].median()) # Заполнение пропусков в 'days_employed'
clients.isna().sum() # проверим результат 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [8]:
# проерим на аномалии другие столбцы
clients.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [9]:
clients['children'].sort_values(ascending=False)


19562    20
11715    20
3302     20
5315     20
13489    20
         ..
16265    -1
6013     -1
21140    -1
19100    -1
15147    -1
Name: children, Length: 21525, dtype: int64

В столбце children также есть отрицательные значения, приведем их к абсолютному значению.

In [10]:
clients['children'] = clients['children'].abs()
clients['children'].value_counts() #проверим распределение занчений в столбце

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Из 21525 заемщиков 76 человек имеют 20 детей. Доля выглядит небольшой (76 / 21525 * 100 = 0,3%), но смущает большой разрыв между предыдущим показателем в пять детей. Поэтому данные о 20 детях, скорее всего, артефакты и от них стоит избавиться. Чтобы не удалять полностью строки с данными о заемщиках, у которых указано 20 детей, заменим данные о детях на NaN.

In [11]:
# заменим данные о 20 детях на NaN
clients.loc[clients['children'] == 20, 'children'] = 'NaN'
clients['children'] = pd.to_numeric(clients['children'], errors='coerce') # заменим тип данных на цифровой
clients['children'].unique() # проверим результат

array([ 1.,  0.,  3.,  2.,  4., nan,  5.])

In [12]:
# проверим на аномалии столбец dob_years, показывающий возраст клиента в годах
clients['dob_years'].sort_values(ascending=False)
display(clients['dob_years'].unique()) # в столбце есть нулевые значения
display(len(clients.query('dob_years == 0'))) # проверим их количество

# проверим, можно ли использовать информацию о стаже, чтобы заполнить нулевые значения в столбце, показывающим возраст клиентов
years = (
    clients[['dob_years','days_employed']]
    .query('dob_years == 0')
    .pivot_table(index = 'dob_years', values = 'days_employed', aggfunc = ['min', 'max', 'median'])
)
years

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

101

,min,max,median
,days_employed,days_employed,days_employed
dob_years,,,
0,108.967042,20805.0,2161.591519


Информация выглядит реалистичной, поэтому заменим нулевые значения на данные, рассчитанные по формуле "трудовой стаж" / 365 дней + 18 лет.

In [13]:
# создадим функцию для замены нулевых значений

def zero_years(row):
    days_employed = row['days_employed']
    dob_years = row['dob_years']
    if dob_years == 0:
        dob_years = days_employed / 365 + 18
    return dob_years

        
clients['dob_years'] = clients.apply(zero_years, axis=1).round(0)
clients['dob_years'].value_counts().reset_index().sort_values(by='index') # проверим результат


,index,dob_years
57,18.0,3
53,19.0,23
50,20.0,59
46,21.0,123
43,22.0,193
41,23.0,261
38,24.0,277
35,25.0,362
33,26.0,408
21,27.0,495


In [14]:
# проверим на аномалии столбец total_income
clients['total_income'].sort_values(ascending=False) # аномалий нет

12412    2.265604e+06
19606    2.200852e+06
9169     1.726276e+06
20809    1.715018e+06
17178    1.711309e+06
             ...     
14276    2.189561e+04
1598     2.169510e+04
16174    2.136765e+04
13006    2.120528e+04
14585    2.066726e+04
Name: total_income, Length: 21525, dtype: float64

In [15]:
# проверим на аномалии столбец debt
clients['debt'].unique() # аномалий нет

array([0, 1])

Столбцы  с типом данных object `education`, `education_id`,`family_status`, `family_status_id`, `gender`, `income_type`, `purpose` мы проверим в следующем пункте.

### Шаг 2.3. Изменение типов данных.

In [16]:
# Заменим вещественный тип данных в столбце 'total_income' на целочисленный
clients['total_income'] = clients['total_income'].astype('int') 


### Шаг 2.4. Удаление дубликатов.

In [17]:
clients.duplicated().sum() # проверим количество дубликатов в таблице

54

In [18]:
clients['education'].unique() # проверим неявные дубликаты в столбце 'education'

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [19]:
clients['education'] = clients['education'].str.lower() # приведем значения к нижнему регистру в столбце 'education'
clients['education'].unique() # проверим результат

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [20]:
clients['family_status'].unique() # проверим неявные дубликаты в столбце 'family_status'

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [21]:
clients['family_status'] = clients['family_status'].str.lower() # приведем значения к нижнему регистру в столбце 'family_status'
clients['family_status'].unique() # дубликатов нет

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [22]:
display(clients['gender'].value_counts()) # проверим неявные дубликаты в столбце 'gender'
# есть аномалия "XNA", но она встречается лишь один раз. Заменим ее на более частотное значение
clients.loc[clients['gender'] == 'XNA', 'gender'] = 'F'

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [23]:
clients['income_type'].unique() # проверим неявные дубликаты в столбце 'income_type'
# дубликатов нет

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [24]:
clients['purpose'].unique() # проверим неявные дубликаты в столбце 'purpose'. Обработаем дубликаты в этом столбце в разделе 7

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [25]:
clients.duplicated().sum() # проверим количество явных дубликатов в таблице еще раз

71

In [26]:
# количество явных дубликатов выросло
clients = clients.drop_duplicates().reset_index(drop=True) # удалим явные дубликаты
clients.duplicated().sum() # проверим результат

0

Для поиска явных дубликатов мы использовали метод `duplicated()`, а неявных — `unique()`. В первом случае мы удалили дубликаты с помощью `drop_duplicates()`, а во втором использовали метод для приведения текста к нижнему регистру `str.lower()`.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
- каждому уникальному значению из `education` соответствует уникальное значение `education_id`;
- каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id`.

In [27]:
education = clients[['education','education_id']] # создадим dataframe, в котором каждому уникальному значению из 'education' соответствует уникальное значение education_id
education = education.drop_duplicates().reset_index(drop=True)
education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [28]:
family = clients[['family_status','family_status_id']] # создадим 'dataframe', в котором каждому уникальному значению из 'family_status' соответствует уникальное значение 'family_status_id'
family = family.drop_duplicates().reset_index(drop=True)
family

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [29]:
# удалим столбцы 'education' и 'family_status'
clients_updated = clients.drop(columns = ['education', 'family_status'])


### Шаг 2.6. Категоризация дохода.
Создадим новый столбец `total_income_category`.

In [31]:
def total_income_category(row):
 
    if row['total_income'] <= 30000:
        return 'E'
    elif 30001 <= row['total_income'] <= 50000:
        return 'D'
    elif  50001 <= row['total_income'] <= 200000:
        return 'C'
    elif 200001 <= row['total_income'] <= 1000000:
        return 'B'
    return 'A'

clients_updated['total_income_category'] = clients_updated.apply(total_income_category, axis=1)
clients_updated.head(10) # проверяем результат

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,8437.673028,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1.0,4024.803754,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0.0,5623.422610,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3.0,4124.747207,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0.0,12775.000000,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0.0,926.185831,27.0,0,1,M,компаньон,0,255763,покупка жилья,B
6,0.0,2879.202052,43.0,0,0,F,компаньон,0,240525,операции с жильем,B
7,0.0,152.779569,50.0,1,0,M,сотрудник,0,135823,образование,C
8,2.0,6929.865299,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0.0,2188.756445,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.
Создадим новый столбец с категориями целей кредита.

In [32]:
def purpose_category(row):
    if 'жил' in row['purpose']:
        return 'операции с недвижимостью'
    elif 'недв' in row['purpose']:
        return 'операции с недвижимостью'
    elif 'свадьб'in row['purpose']:
        return 'проведение свадьбы'
    elif 'образ' in row['purpose']:
        return 'получение образования'
    elif 'авто' in row['purpose']:
        return 'операции с автомобилем'
    return row['purpose']

clients_updated['purpose_category'] = clients_updated.apply(purpose_category, axis=1)
clients_updated['purpose_category'].unique() # проверим результат

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Ответы на вопросы.

#### Вопрос 1: 
Есть ли зависимость между количеством детей и возвратом кредита в срок?
#### Ответ 1:
Согласно вычислениям ниже, процент тех, кто не возвращает кредит в срок, выше среди имеющих 4, 2 и 1 ребенка (9 процентов). Однако эти показатели не намного выше тех, которые мы рассчитали для людей, не имеющих детей (7 процентов). Из этого можно сделать вывод, что наличие детей практически не влияет на факт возврата кредита в срок.


In [54]:
# создадим сводную таблицу с данными столбцов 'debt' и 'children'
# рассчитаем соотношение тех, кто просрочил выплату по кредиту к тем, кто не просрочил
children_pivot = (
    clients_updated
    .pivot_table(index='children', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
    .reset_index()
    .rename(columns={'debt': '', 'sum': 'Кол-во просрочек', 'count': 'Общее кол-во', 'mean': 'Доля', 'children': 'Кол-во детей'})
)
children_pivot

,Кол-во детей,Кол-во просрочек,Общее кол-во,Доля
,,,,
0,0.0,1063,14091,0.075438
1,1.0,445,4855,0.091658
2,2.0,194,2052,0.094542
3,3.0,27,330,0.081818
4,4.0,4,41,0.097561
5,5.0,0,9,0.000000


#### Вопрос 2: 
Есть ли зависимость между семейным положением и возвратом кредита в срок?
#### Ответ 2:
Согласно вычислениям ниже, есть небольшая зависимость между семейным статусом и фактом возврата кредита в срок. Заемщики в разводе или статусе "вдовец/вдова" не возвращают кредит в срок только в 7% случаев, а заемщики в гражданском браке или в статусе "не женат / не замужем" — в 10% случаев. Женатые/замужние заемщики не возвращают кредит вовремя в 7%. Влияние семейного положения есть, но небольшое.

In [64]:
# создадим сводную таблицу с данными столбцов 'debt' и 'family_status_id'
# рассчитаем соотношение тех, кто просрочил выплату кредита к тем, кто не просрочил
family_pivot = (
    clients_updated.pivot_table(index='family_status_id', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
    .reset_index()
    .rename(columns={'debt': '', 'sum': 'Кол-во просрочек', 'count': 'Общее кол-во', 'mean': 'Доля', 'family_status_id': 'Статус'})
    )

# заменим family_status_id на название статуса

fam_stat_dict = {0: 'женат / замужем', 1: 'гражданский брак', 2: 'вдовец / вдова', 3: 'в разводе', 4: 'не женат / не замужем'}
family_pivot['Статус'] = family_pivot['Статус'].map(fam_stat_dict)

family_pivot


,Статус,Кол-во просрочек,Общее кол-во,Доля
,,,,
0,женат / замужем,931,12339,0.075452
1,гражданский брак,388,4151,0.093471
2,вдовец / вдова,63,959,0.065693
3,в разводе,85,1195,0.071130
4,не женат / не замужем,274,2810,0.097509


#### Вопрос 3: 
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
#### Ответ 3:
Согласно данным ниже, заемщики категории D (30001–50000) меньше всех затягивают выплаты кредита, т.е. являются наиболее примерными плательщиками. Серьезных корреляций в остальных категориях не выявлено, средний процент невозврата — около 8.


In [66]:
# создадим сводную таблицу с данными столбцов 'debt' и 'total_income_category'
income_pivot = (
    clients_updated
    .pivot_table(index='total_income_category', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
    .reset_index()
    .rename(columns={'debt': '', 'sum': 'Кол-во просрочек', 'count': 'Общее кол-во', 'mean': 'Доля', 'total_income_category': 'Категория'})
)

income_pivot

,Категория,Кол-во просрочек,Общее кол-во,Доля
,,,,
0,A,2,25,0.080000
1,B,356,5041,0.070621
2,C,1360,16016,0.084915
3,D,21,350,0.060000
4,E,2,22,0.090909


#### Вопрос 4: 
Как разные цели кредита влияют на его возврат в срок?
#### Ответ 4:
Чаще не выплачивают кредит в срок заемщики, получившие кредит на цели "операции с автомобилем" (9%) и "получение образования" (9%), реже всего — в категории "операции с недвижимостью" (7%). Однако разница между категориями в 2 процента не кажется значительной.

In [67]:
# создадим сводную таблицу с данными столбцов 'debt' и 'purpose_category'
purpose_pivot = (
    clients_updated
    .pivot_table(index='purpose_category', values = 'debt', aggfunc = ['sum', 'count', 'mean'])
    .reset_index()
    .rename(columns={'debt': '', 'sum': 'Кол-во просрочек', 'count': 'Общее кол-во', 'mean': 'Доля', 'purpose_category': 'Категория'})
)
purpose_pivot 

,Категория,Кол-во просрочек,Общее кол-во,Доля
,,,,
0,операции с автомобилем,403,4306,0.093590
1,операции с недвижимостью,782,10811,0.072334
2,получение образования,370,4013,0.092200
3,проведение свадьбы,186,2324,0.080034


#### Вопрос 5: 
Ответы сопроводите интерпретацией — поясните, о чём именно говорит полученный вами результат.
#### Ответ 5:
Полученный результат позволяет сделать вывод о том, что серьезной зависимости между возвратом кредита в срок и наличием детей/семейным статусом/уровнем дохода/целями получения кредита нет.

## Общий вывод:

Для построения модели кредитного скоринга банка необходимо было разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные — статистика о платёжеспособности клиентов.

В ходе исследования мы проверили данные на ошибки и оценили их влияние на исследование. В данных были обнаружены пропуски, артефакты, а также явные и неявные дубликаты.

После устранения проблем в данных:
* заполнения пропущенных данных  в столбцах `days_employed` и `total_income` (доля пропущенных значений в каждом из столбцов с пропусками составляет 10%) медианными значениями, 
* обработки артефактов:
   * в столбце `days_employed` оказались отрицательные значения дней трудового стажа, а также нереалистичное количество трудовых дней,
   * в столбце `children` оказались отрицательные значения и 76 заемщиков с 20 детьми,
   * в столбце `dob_years` оказались нулевые данные
* устранения дубликатов, 
* категоризации данных

мы проверили четыре  гипотезы:
1. Существует зависимость между количеством детей и возвратом кредита в срок.
2. Существует зависимость между семейным положением и возвратом кредита в срок.
3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
4. Разные цели кредита по-разному влияют на его возврат в срок. 

Полученный на основании анализа данных результат позволяет сделать вывод о том, что наличие детей / семейный статус / уровень дохода / цель получения кредита не оказывает серьезного влияния на кредитоспособность заемщика, то есть вероятность возврата или невозврата кредита. Для построении модели кредитного скоринга мы рекомендуем изучить дополнительные характеристики потенциальных заемщиков для определения их кредитоспособности. 